In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import nltk
import string
import re
from os import path
from PIL import Image

from spellchecker import SpellChecker
from nltk import FreqDist
from collections import Counter
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

import gensim

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from textblob import TextBlob

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
/home/pablo/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pablo/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pablo/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pablo/anaconda3/lib/python3.7/site-packages/tensorflo

In [3]:
stop_words = stopwords.words('english')

In [4]:
df_train = pd.read_csv("dataset/train.csv")
df_test= pd.read_csv("dataset/test.csv")
pd.set_option('display.max_colwidth', -1) #elimino limite de truncado para visualizacion

In [4]:
#selecciono los ids con target erroneo
ids = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
df_train.loc[df_train['id'].isin(ids),'target'] = 0

## Construccion de Features

In [5]:
# contador de palabras
df_train['word_count'] = df_train['text'].apply(lambda x: len(str(x).split()))
df_test['word_count'] = df_test['text'].apply(lambda x: len(str(x).split()))

# palabras unicas
df_train['unique_word_count'] = df_train['text'].apply(lambda x: len(set(str(x).split())))
df_test['unique_word_count'] = df_test['text'].apply(lambda x: len(set(str(x).split())))

# contador de stopwords
df_train['stop_word_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))
df_test['stop_word_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))

# contador de url
df_train['url_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
df_test['url_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# extractor de emails
df_train['emails'] = df_train['text'].apply(lambda x: re.findall(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9+._-]+\.[a-zA-Z0-9+._-]+)', x))
df_test['emails'] = df_test['text'].apply(lambda x: re.findall(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9+._-]+\.[a-zA-Z0-9+._-]+)', x))

# contador de emails
df_train['emails_count'] = df_train['emails'].apply(lambda x: len(x))
df_test['emails_count'] = df_test['emails'].apply(lambda x: len(x))

# promedio del largo de palabras
df_train['mean_word_length'] = df_train['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test['mean_word_length'] = df_test['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# contador de caracteres
df_train['char_count'] = df_train['text'].apply(lambda x: len(str(x)))
df_test['char_count'] = df_test['text'].apply(lambda x: len(str(x)))

# contador de numeros
df_train['number_count'] = df_train['text'].apply(lambda x: len([w for w in x.split() if w.isdigit()]))
df_test['number_count'] = df_test['text'].apply(lambda x: len([w for w in x.split() if w.isdigit()]))

# contador de mayuscuslas en palabras 
df_train['upper_count'] = df_train['text'].apply(lambda x: len([w for w in x.split() if w.isupper() and len(x)>3]))
df_test['upper_count'] = df_test['text'].apply(lambda x: len([w for w in x.split() if w.isupper() and len(x)>3]))

# contador de puntuaciones
df_train['punctuation_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test['punctuation_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# contador de hashtags
df_train['hashtag_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
df_test['hashtag_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# contador de menciones
df_train['mention_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
df_test['mention_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

## Limpieza del Texto

In [6]:
# con el fin de agilizar el tiempo para la construccion de algunos filtros se han recolectado de diferentes fuentes

def limpieza(text):
            
    # elimino caracteres especiales
    text = re.sub(r"\x89Û_", "", text)
    text = re.sub(r"\x89ÛÒ", "", text)
    text = re.sub(r"\x89ÛÓ", "", text)
    text = re.sub(r"\x89ÛÏWhen", "When", text)
    text = re.sub(r"\x89ÛÏ", "", text)
    text = re.sub(r"China\x89Ûªs", "China's", text)
    text = re.sub(r"let\x89Ûªs", "let's", text)
    text = re.sub(r"\x89Û÷", "", text)
    text = re.sub(r"\x89Ûª", "", text)
    text = re.sub(r"\x89Û\x9d", "", text)
    text = re.sub(r"å_", "", text)
    text = re.sub(r"\x89Û¢", "", text)
    text = re.sub(r"\x89Û¢åÊ", "", text)
    text = re.sub(r"fromåÊwounds", "from wounds", text)
    text = re.sub(r"åÊ", "", text)
    text = re.sub(r"åÈ", "", text)
    text = re.sub(r"JapÌ_n", "Japan", text)    
    text = re.sub(r"Ì©", "e", text)
    text = re.sub(r"å¨", "", text)
    text = re.sub(r"SuruÌ¤", "Suruc", text)
    text = re.sub(r"åÇ", "", text)
    text = re.sub(r"å£3million", "3 million", text)
    text = re.sub(r"åÀ", "", text)
    
    # Reemplazo contracciones
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"there's", "there is", text)
    text = re.sub(r"We're", "We are", text)
    text = re.sub(r"That's", "That is", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"they're", "they are", text)
    text = re.sub(r"Can't", "Cannot", text)
    text = re.sub(r"wasn't", "was not", text)
    text = re.sub(r"don\x89Ûªt", "do not", text)
    text = re.sub(r"aren't", "are not", text)
    text = re.sub(r"isn't", "is not", text)
    text = re.sub(r"What's", "What is", text)
    text = re.sub(r"haven't", "have not", text)
    text = re.sub(r"hasn't", "has not", text)
    text = re.sub(r"There's", "There is", text)
    text = re.sub(r"He's", "He is", text)
    text = re.sub(r"It's", "It is", text)
    text = re.sub(r"You're", "You are", text)
    text = re.sub(r"I'M", "I am", text)
    text = re.sub(r"shouldn't", "should not", text)
    text = re.sub(r"wouldn't", "would not", text)
    text = re.sub(r"i'm", "I am", text)
    text = re.sub(r"I\x89Ûªm", "I am", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r"Isn't", "is not", text)
    text = re.sub(r"Here's", "Here is", text)
    text = re.sub(r"you've", "you have", text)
    text = re.sub(r"you\x89Ûªve", "you have", text)
    text = re.sub(r"we're", "we are", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"we've", "we have", text)
    text = re.sub(r"it\x89Ûªs", "it is", text)
    text = re.sub(r"doesn\x89Ûªt", "does not", text)
    text = re.sub(r"It\x89Ûªs", "It is", text)
    text = re.sub(r"Here\x89Ûªs", "Here is", text)
    text = re.sub(r"who's", "who is", text)
    text = re.sub(r"I\x89Ûªve", "I have", text)
    text = re.sub(r"y'all", "you all", text)
    text = re.sub(r"can\x89Ûªt", "cannot", text)
    text = re.sub(r"would've", "would have", text)
    text = re.sub(r"it'll", "it will", text)
    text = re.sub(r"we'll", "we will", text)
    text = re.sub(r"wouldn\x89Ûªt", "would not", text)
    text = re.sub(r"We've", "We have", text)
    text = re.sub(r"he'll", "he will", text)
    text = re.sub(r"Y'all", "You all", text)
    text = re.sub(r"Weren't", "Were not", text)
    text = re.sub(r"Didn't", "Did not", text)
    text = re.sub(r"they'll", "they will", text)
    text = re.sub(r"they'd", "they would", text)
    text = re.sub(r"DON'T", "DO NOT", text)
    text = re.sub(r"That\x89Ûªs", "That is", text)
    text = re.sub(r"they've", "they have", text)
    text = re.sub(r"i'd", "I would", text)
    text = re.sub(r"should've", "should have", text)
    text = re.sub(r"You\x89Ûªre", "You are", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"Don\x89Ûªt", "Do not", text)
    text = re.sub(r"we'd", "we would", text)
    text = re.sub(r"i'll", "I will", text)
    text = re.sub(r"weren't", "were not", text)
    text = re.sub(r"They're", "They are", text)
    text = re.sub(r"Can\x89Ûªt", "Cannot", text)
    text = re.sub(r"you\x89Ûªll", "you will", text)
    text = re.sub(r"I\x89Ûªd", "I would", text)
    text = re.sub(r"let's", "let us", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"you're", "you are", text)
    text = re.sub(r"i've", "I have", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"i'll", "I will", text)
    text = re.sub(r"doesn't", "does not", text)
    text = re.sub(r"i'd", "I would", text)
    text = re.sub(r"didn't", "did not", text)
    text = re.sub(r"ain't", "am not", text)
    text = re.sub(r"you'll", "you will", text)
    text = re.sub(r"I've", "I have", text)
    text = re.sub(r"Don't", "do not", text)
    text = re.sub(r"I'll", "I will", text)
    text = re.sub(r"I'd", "I would", text)
    text = re.sub(r"Let's", "Let us", text)
    text = re.sub(r"you'd", "You would", text)
    text = re.sub(r"It's", "It is", text)
    text = re.sub(r"Ain't", "am not", text)
    text = re.sub(r"Haven't", "Have not", text)
    text = re.sub(r"Could've", "Could have", text)
    text = re.sub(r"youve", "you have", text)  
    text = re.sub(r"donå«t", "do not", text)    
    
    return text


df_train['text']=df_train['text'].apply(lambda x: limpieza(x))
df_test['text']=df_test['text'].apply(lambda x: limpieza(x))


In [7]:
def eliminacion(text):
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
     
    ## clean urls 
    url = re.compile(r'https?://\S+|www\.\S+')
    text = url.sub(r'',text)
    
    url = re.compile(r'http?://\S+|www\.\S+')
    text = url.sub(r'',text)
    
    ## remove html 
    html=re.compile(r'<.*?>') 
    html.sub(r'',text)
    
    return text


df_train['text']=df_train['text'].apply(lambda x: eliminacion(x))
df_test['text']=df_test['text'].apply(lambda x: eliminacion(x))


In [8]:
def toke_lemma(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    # Tokenizador
    lst_text = text.split()
    ## elimino stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    # Stemming 
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    # Lemma
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
            
    text = " ".join(lst_text)
    
    return text


df_train['text']=df_train['text'].apply(lambda x: toke_lemma(x, flg_stemm=False, flg_lemm=True, lst_stopwords = stop_words))
df_test['text']=df_test['text'].apply(lambda x: toke_lemma(x, flg_stemm=False, flg_lemm=True, lst_stopwords = stop_words))

In [9]:
df = pd.concat([df_train,df_test])

In [10]:
corpus = df['text']

In [11]:
#convierto a secuencias
embedding_dict={}
#with open('dataset/glove.twitter.27B.100d.txt','r') as f:
with open('dataset/glove.42B.300d.txt','r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

In [12]:
MAX_LEN=50
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

In [13]:
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))
#19459
#18675


Number of unique words: 27191


In [14]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,300))

for word,i in word_index.items():
    if i > num_words:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec

## Modelo Base

In [15]:
model=Sequential()

embedding=Embedding(num_words,300,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)

model.add(embedding)
model.add(SpatialDropout1D(0.1))
model.add(LSTM(300, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(1, activation='sigmoid'))


optimzer=Adam(learning_rate=3e-4)

model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 300)           8157600   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 50, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 301       
Total params: 8,879,101
Trainable params: 721,501
Non-trainable params: 8,157,600
_________________________________________________________________


In [17]:
train=tweet_pad[:df_train.shape[0]]
test=tweet_pad[df_train.shape[0]:]

In [18]:
X_train,X_test,y_train,y_test=train_test_split(train,df_train['target'].values,test_size=0.2)
print('Shape of train',X_train.shape)
print("Shape of Validation ",X_test.shape)

Shape of train (6090, 50)
Shape of Validation  (1523, 50)


## Entro el modelo

In [19]:
history=model.fit(X_train,y_train,batch_size=128,epochs=25,validation_data=(X_test,y_test),verbose=2)

Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Train on 6090 samples, validate on 1523 samples
Epoch 1/25
 - 64s - loss: 0.5657 - accuracy: 0.7057 - val_loss: 0.5409 - val_accuracy: 0.7708
Epoch 2/25
 - 63s - loss: 0.4594 - accuracy: 0.8033 - val_loss: 0.5044 - val_accuracy: 0.7873
Epoch 3/25
 - 63s - loss: 0.4411 - accuracy: 0.8135 - val_loss: 0.4759 - val_accuracy: 0.7800
Epoch 4/25
 - 63s - loss: 0.4306 - accuracy: 0.8190 - val_loss: 0.4778 - val_accuracy: 0.7807
Epoch 5/25
 - 63s - loss: 0.4230 - accuracy: 0.8209 - val_loss: 0.4919 - val_accuracy: 0.7768
Epoch 6/25
 - 63s - loss: 0.4192 - accuracy: 0.8258 - val_loss: 0.4954 - val_accuracy: 0.7820
Epoch 7/25
 - 63s - loss: 0.4100 - accuracy: 0.8314 - val_loss: 0.4725 - val_accuracy: 0.7853
Epoch 8/25
 - 68s - loss: 0.4074 - accuracy: 0.8325 - val_loss: 0.4766 - val_accuracy: 0.7807
Epoch 9/25
 - 63s - loss: 0.4050 - accuracy: 0.8342 - val_loss: 0.4621 - val_accuracy: 0.7879
Epoch 10/25
 - 63s - loss: 0.3997 - accuracy: 0.8309 - val_loss: 0.4881 - val_accuracy: 0.7794
Epoch 11/25

In [20]:
train_pred_GloVe = model.predict(train)
train_pred_GloVe_int = train_pred_GloVe.round().astype('int')

In [21]:
test_pred_GloVe = model.predict(test)
test_pred_GloVe_int = test_pred_GloVe.round().astype('int')

submission = pd.read_csv("dataset/sample_submission.csv")
submission['target'] = test_pred_GloVe_int
submission.head(10)

submission.to_csv("submission10.csv", index=False, header=True)
